In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
import os
print(os.listdir("../input"))
from fastai.imports import *
# Any results you write to the current directory are saved as output.

['Data_Dictionary.xlsx', 'train-mahindra', 'sample_submission_dlc0jkw', 'test_mahindra']


### Read the data

In [2]:
train = pd.read_csv('../input/train-mahindra/train.csv')
test = pd.read_csv('../input/test_mahindra/test.csv')
submit = pd.read_csv('../input/sample_submission_dlc0jkw/sample_submission.csv')

### Helper functions for feature engineering

In [3]:
## To display all the data 
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [4]:
## add columns relevant in a date column
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [47]:
train.shape, test.shape, train.shape[0] + test.shape[0], submit.shape

((341424, 67), (146765, 67), 488189, (146765, 2))

In [6]:
display_all(train.head())

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,7.706428
1,03930f033646d073462b35d411616323597715ac4fc398...,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,6.662563
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,28/01/15,01/02/15,05/02/15,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,7.871602
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,02/05/15,11/06/15,16/06/15,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,5.344943
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,02/09/15,14/12/15,19/12/15,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,7.059346


In [7]:
test.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id
0,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d...,05/04/18,05/04/18,06/04/18,3,2,2,0,45,3,3,4,1,2.0,2.0,3,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,A,C,9f14025af0065b30e47e23ebb3b491d39ae8ed17d33739...
1,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb...,24/05/15,23/06/15,27/06/15,1,2,4,0,45,1,1,4,8,2.0,2.0,4,2,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,F,A,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...
2,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e...,21/07/15,07/08/15,10/08/15,3,2,3,0,45,2,2,2,3,4.0,2.0,2,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...
3,09593c907ae262e46f655b4db9e14f54a19eadcfdd2679...,31/07/15,22/10/15,25/10/15,1,2,3,0,45,1,1,2,3,2.0,2.0,6,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,F,A,49d180ecf56132819571bf39d9b7b342522a2ac6d23c14...
4,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1...,01/07/16,01/10/16,05/10/16,1,2,4,0,45,1,2,3,8,2.0,2.0,11,2,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,F,A,e29c9c180c6279b0b02abd6a1801c7c04082cf486ec027...


In [8]:
submit.head()

,reservation_id,amount_spent_per_room_night_scaled
0,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d...,8
1,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb...,8
2,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e...,8
3,09593c907ae262e46f655b4db9e14f54a19eadcfdd2679...,8
4,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1...,8


In [9]:
## lets check the datatype of the columns
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341424 entries, 0 to 341423
Data columns (total 24 columns):
reservation_id                        341424 non-null object
booking_date                          341424 non-null object
checkin_date                          341424 non-null object
checkout_date                         341424 non-null object
channel_code                          341424 non-null int64
main_product_code                     341424 non-null int64
numberofadults                        341424 non-null int64
numberofchildren                      341424 non-null int64
persontravellingid                    341424 non-null int64
resort_region_code                    341424 non-null int64
resort_type_code                      341424 non-null int64
room_type_booked_code                 341424 non-null int64
roomnights                            341424 non-null int64
season_holidayed_code                 341310 non-null float64
state_code_residence                  33666

In [10]:
## check for missing values in a column
train.isnull().sum(), test.isnull().sum()

(reservation_id                           0
 booking_date                             0
 checkin_date                             0
 checkout_date                            0
 channel_code                             0
 main_product_code                        0
 numberofadults                           0
 numberofchildren                         0
 persontravellingid                       0
 resort_region_code                       0
 resort_type_code                         0
 room_type_booked_code                    0
 roomnights                               0
 season_holidayed_code                  114
 state_code_residence                  4764
 state_code_resort                        0
 total_pax                                0
 member_age_buckets                       0
 booking_type_code                        0
 memberid                                 0
 cluster_code                             0
 reservationstatusid_code                 0
 resort_id                      

In [11]:
## let's introduce target column in test as well for easier feature engineering
#y = train.amount_spent_per_room_night_scaled
#X = train.drop(['amount_spent_per_room_night_scaled'], axis = 1)
test['amount_spent_per_room_night_scaled'] = np.zeros

In [12]:
## give label to data based on train/test and concat both data set
train['type'] = 'train'
test['type'] = 'test'
data = pd.concat([train, test], axis = 0, ignore_index= True)  

In [13]:
## helper function to fix missing values
## creates an extra column indicating if that row is missing from the data or not
def fix_missing(df, col, name, na_dict):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name+'_na'] = pd.isnull(col)
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict

In [14]:
## let's fix the missing values
fix_missing(data, data['season_holidayed_code'], 'season_holidayed_code', {})
fix_missing(data, data['state_code_residence'], 'state_code_residence', {})

{'state_code_residence': 8.0}

 ### Feature engineering

In [15]:
data['total_people'] = data['numberofadults'] + data['numberofchildren']
data['total_adult_night'] = data['numberofadults'] * data['roomnights']
data['total_people_night'] = data['total_people'] * data['roomnights']

In [16]:
## convert data columns to date format
# data['booking_date'] = pd.to_datetime(data['booking_date'], format= '%d/%m/%y')
# data['checkin_date'] = pd.to_datetime(data['checkin_date'], format= '%d/%m/%y')
# data['checkout_date'] = pd.to_datetime(data['checkout_date'], format= '%d/%m/%y')

## advance booking days
data['days_advance']=(pd.to_datetime(data['checkin_date'],format= '%d/%m/%y')-pd.to_datetime(data['booking_date'],format= '%d/%m/%y')).dt.days

## actual stays based on checkin and checkout
data['actual_stay']=(pd.to_datetime(data['checkout_date'],format= '%d/%m/%y')-pd.to_datetime(data['checkin_date'],format= '%d/%m/%y')).dt.days

In [17]:
## lets see the statistics
data.describe()

,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,booking_type_code,total_people,total_adult_night,total_people_night,days_advance,actual_stay
count,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000
mean,1.902587,2.207459,3.275522,0.362573,209.955489,1.773303,2.194687,2.889850,3.735949,2.676134,7.269420,4.759472,3.191893,1.123952,3.638095,14.750271,16.280447,46.119677,2.466323
std,0.915112,0.821989,1.764458,0.758078,864.891408,0.751550,1.549417,0.804194,2.480057,0.884552,4.684542,3.103048,1.166638,0.329527,1.888372,26.392213,27.796234,38.693097,1.094641
min,1.000000,1.000000,0.000000,0.000000,45.000000,1.000000,0.000000,1.000000,-45.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,-180.000000,-315.000000,-2219.000000,1.000000
25%,1.000000,2.000000,2.000000,0.000000,45.000000,1.000000,1.000000,2.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,2.000000,6.000000,6.000000,15.000000,2.000000
50%,2.000000,2.000000,3.000000,0.000000,45.000000,2.000000,2.000000,3.000000,3.000000,3.000000,8.000000,5.000000,3.000000,1.000000,3.000000,8.000000,9.000000,33.000000,2.000000
75%,3.000000,3.000000,4.000000,0.000000,45.000000,2.000000,3.000000,3.000000,4.000000,3.000000,8.000000,7.000000,4.000000,1.000000,4.000000,16.000000,16.000000,82.000000,3.000000
max,3.000000,7.000000,32.000000,13.000000,4995.000000,3.000000,7.000000,6.000000,80.000000,4.000000,38.000000,13.000000,24.000000,2.000000,34.000000,1536.000000,1536.000000,177.000000,26.000000


In [18]:
## room night has a minimum value which is negative. lets see all the rows
data[data['roomnights'] < 0]

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled,type,season_holidayed_code_na,state_code_residence_na,total_people,total_adult_night,total_people_night,days_advance,actual_stay
36008,8cf7476b7111e0f969ef00d582f8a0833794239ebd3206...,30/12/14,22/03/15,29/03/15,1,2,4,3,45,1,1,2,-45,3.0,8.0,6,4,D,1,a3af543aae775ab3cac3f789f74d5c0a70e6c0ecacb0db...,F,A,b17ef6d19c7a5b1ee83b907c595526dcb1eb06db8227d6...,7.77926,train,False,False,7,-180,-315,82,7


In [19]:
## its just a row.  replace the values manually based on previus calculation  
data['roomnights'] = data['roomnights'].replace(-45, 7)
data['total_adult_night'] = data['total_adult_night'].replace(-180, 28)
data['total_people_night'] = data['total_people_night'].replace(-315, 49)

In [20]:
## we also see that days advance is negative for some roows. lets see them
data[data['days_advance'] < 0]

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled,type,season_holidayed_code_na,state_code_residence_na,total_people,total_adult_night,total_people_night,days_advance,actual_stay
15,821a5e5313029b1bbec6ec738a1b34d6d14c82fca97c47...,05/04/18,08/03/12,11/03/12,3,1,2,0,45,1,1,2,1,3.0,8.0,6,3,H,1,7b68caeb153ac83aeab0b9a679709c558d57a6c9d2bdf6...,F,C,49d180ecf56132819571bf39d9b7b342522a2ac6d23c14...,3.21888,train,False,False,2,2,2,-2219,3
34,a80d2fbea1195021307a13a44642855c60f46145dcf29c...,31/03/18,12/03/12,18/03/12,3,2,3,0,45,1,1,3,2,2.0,5.0,1,3,D,2,d4eee99d5c3238cdda52479fb34125e15208d394af34e7...,E,C,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,7.02972,train,False,False,3,6,6,-2210,6
60,51fe7242f55c79aacb499f957c746445ccb51de3c8d206...,05/04/18,24/03/12,25/03/12,3,2,2,0,45,3,3,3,1,2.0,6.0,3,4,I,1,a24896d1582d8b6c0de997d8b7435962e33580ba80839d...,A,C,f5ca38f748a1d6eaf726b8a42fb575c3c71f1864a81433...,9.14201,train,False,False,2,2,2,-2203,1
79,548605b68da37dd5c8555a37650910ce9f110e793b4619...,05/04/18,21/03/12,25/03/12,1,3,3,1,45,3,3,2,6,2.0,4.0,5,3,E,1,3bb1aa01c771c2217fe08bae162569a3177da7d09ae6be...,F,A,7902699be42c8a8e46fbbb4501726517e86b22c56a189f...,7.2529,train,False,False,4,18,24,-2206,4
81,3ac8796577c899cd0177b6f9638515e4feab55d8a72b8b...,05/04/18,23/03/12,26/03/12,1,3,4,0,45,1,1,3,4,2.0,2.0,1,3,E,1,fa856d4dde8f88ed2855a74e2468cedd802ee253434a6c...,E,C,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,7.22954,train,False,False,4,16,16,-2204,3
113,4d73a3304416c4bff78b47c6b307a07568d7286d946e3e...,06/04/18,09/04/12,11/04/12,1,2,3,1,45,1,2,2,4,3.0,1.0,11,6,D,1,c2eaeada5db1e75552e54382c408fe1c600679060b052c...,F,C,e29c9c180c6279b0b02abd6a1801c7c04082cf486ec027...,6.51639,train,False,False,4,12,16,-2188,2
116,8e7c40db97f7088ea2249bb9da85b1cfce31c8b537dcfa...,06/04/18,14/04/12,15/04/12,1,3,4,0,45,1,2,2,2,3.0,1.0,11,4,G,1,48a9cdd22b472fe4aa225acd45acdbe246de12fade4a59...,F,C,e29c9c180c6279b0b02abd6a1801c7c04082cf486ec027...,8.81959,train,False,False,4,8,8,-2183,1
129,085264337c2af42038a81b4ec4d9e278d9699e5662d404...,06/04/18,19/04/12,21/04/12,3,1,2,0,45,3,3,4,1,2.0,5.0,5,3,B,1,914d3756e6da218a071578d46a3a3c4fce3b773a1736cb...,F,C,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,8.24644,train,False,False,2,2,2,-2178,2
130,7294817a3becfb22650a6e4cf0dd7d2a3a1a3969e1e01d...,06/04/18,19/04/12,21/04/12,3,1,2,0,45,3,3,4,1,2.0,5.0,5,3,B,1,914d3756e6da218a071578d46a3a3c4fce3b773a1736cb...,F,C,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,3.19358,train,False,False,2,2,2,-2178,2
147,e36df886dd8222cad63291754719419d5ff50b44e525e0...,06/04/18,22/04/12,26/04/12,3,3,4,0,45,1,1,3,1,3.0,9.0,6,3,B,2,0beacf60d9f72cdfe3f4b8f1f439c605b9bc37ca3a3922...,F,A,b17ef6d19c7a5b1ee83b907c595526dcb1eb06db8227d6...,7.01701,train,False,False,4,4,4,-2175,4


In [21]:
## it seems like there may be an error in these rows. lets replace those by 0 anyway
data[data['days_advance']<0]['days_advance'] = 0

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
## create a column overstay to represent if the actual stay (based on checkin and check out date) is greater or less or equal to 
## given in the data
conditions = [
    (data['actual_stay'] > data['roomnights']),
    (data['actual_stay'] < data['roomnights']),
    (data['actual_stay'] == data['roomnights'])]
choices = ['overstay', 'understay', 'rightstay']
data['over_stay'] = np.select(conditions, choices)

In [23]:
## we see that advance booking has been done prior to 177 days. lets group these days to fom a different column
bins = [-0.01, 0.01, 7, 15, 30, 60, 90, 120, 180, 210]
names = ['groupA', 'groupB', 'groupC', 'groupD', 'groupE', 'groupF','groupG', 'groupH', 'groupI']
data['advance_booking_group'] = pd.cut(data['days_advance'], bins, labels=names)

In [24]:
data.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled,type,season_holidayed_code_na,state_code_residence_na,total_people,total_adult_night,total_people_night,days_advance,actual_stay,over_stay,advance_booking_group
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,7.70643,train,False,False,2,2,2,0,1,rightstay,groupA
1,03930f033646d073462b35d411616323597715ac4fc398...,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,6.66256,train,False,False,2,10,10,78,5,rightstay,groupF
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,28/01/15,01/02/15,05/02/15,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,7.8716,train,False,False,2,8,8,4,4,rightstay,groupB
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,02/05/15,11/06/15,16/06/15,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,5.34494,train,False,False,4,10,20,40,5,rightstay,groupE
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,02/09/15,14/12/15,19/12/15,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,7.05935,train,False,False,2,10,10,103,5,rightstay,groupG


In [25]:
data.shape, train.shape[0]+test.shape[0]

((488189, 34), 488189)

In [26]:
## add date information columns
add_datepart(data, 'booking_date')
add_datepart(data, 'checkin_date')
add_datepart(data, 'checkout_date')

In [27]:
## label encode the object columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['resort_id'] = le.fit_transform(data['resort_id'])
data['memberid'] = le.fit_transform(data['memberid'])

In [28]:
## convert these below object columns to category as model  handles this type much better
for col in [ 'member_age_buckets', 'cluster_code', 'reservationstatusid_code','over_stay','season_holidayed_code_na',
           'state_code_residence_na','memberid','resort_id']:
    data[col] = data[col].astype('category')

In [29]:
data.shape

(488189, 70)

In [30]:
## split the data to train and test set
## define y (target) column
train = data[data['type'] == 'train']
test = data[data['type'] == 'test']
y = train['amount_spent_per_room_night_scaled']

## delete the  id columns which are not part of the model
train_id = train.reservation_id
test_id = test.reservation_id
train = train.drop(['type','amount_spent_per_room_night_scaled','reservation_id'], axis = 1)
test = test.drop(['type','amount_spent_per_room_night_scaled','reservation_id'], axis = 1)


In [31]:
train.shape, y.shape,test.shape, train.shape[0]+test.shape[0]

((341424, 67), (341424,), (146765, 67), 488189)

In [32]:
max(y)

10.81665194

In [33]:
## define rmse as per problem statement
def rmse(x,y): return math.sqrt(((x-y)**2).mean()) * 100

In [34]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341424 entries, 0 to 341423
Data columns (total 67 columns):
channel_code                 341424 non-null int64
main_product_code            341424 non-null int64
numberofadults               341424 non-null int64
numberofchildren             341424 non-null int64
persontravellingid           341424 non-null int64
resort_region_code           341424 non-null int64
resort_type_code             341424 non-null int64
room_type_booked_code        341424 non-null int64
roomnights                   341424 non-null int64
season_holidayed_code        341424 non-null float64
state_code_residence         341424 non-null float64
state_code_resort            341424 non-null int64
total_pax                    341424 non-null int64
member_age_buckets           341424 non-null category
booking_type_code            341424 non-null int64
memberid                     341424 non-null category
cluster_code                 341424 non-null category
reservati

In [35]:
train.columns

Index(['channel_code', 'main_product_code', 'numberofadults',
       'numberofchildren', 'persontravellingid', 'resort_region_code',
       'resort_type_code', 'room_type_booked_code', 'roomnights',
       'season_holidayed_code', 'state_code_residence', 'state_code_resort',
       'total_pax', 'member_age_buckets', 'booking_type_code', 'memberid',
       'cluster_code', 'reservationstatusid_code', 'resort_id',
       'season_holidayed_code_na', 'state_code_residence_na', 'total_people',
       'total_adult_night', 'total_people_night', 'days_advance',
       'actual_stay', 'over_stay', 'advance_booking_group', 'booking_Year',
       'booking_Month', 'booking_Week', 'booking_Day', 'booking_Dayofweek',
       'booking_Dayofyear', 'booking_Is_month_end', 'booking_Is_month_start',
       'booking_Is_quarter_end', 'booking_Is_quarter_start',
       'booking_Is_year_end', 'booking_Is_year_start', 'booking_Elapsed',
       'checkin_Year', 'checkin_Month', 'checkin_Week', 'checkin_Day',
     

In [51]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import StratifiedKFold, KFold
def k_folds_lgb(X, y, X_test, k,n_est,num_leaves,lr):
    folds = KFold( n_splits = k, shuffle=True, random_state=2)
    y_test = 0
    y_oof = np.zeros((X.shape[0]))
    score = 0
    for i, (train_idx, val_idx) in  enumerate(folds.split(X, y)):
        clf =  LGBMRegressor(random_state =1, n_estimators = n_est, n_jobs = -1,learning_rate=lr, num_leaves=num_leaves,reg_alpha=0.1, 
                             reg_lambda=0.1, categorical_feature = [13,15,16,17,18,19,20,26,27])
        clf.fit(X.iloc[train_idx], y[train_idx])
        y_oof[val_idx] = np.clip(clf.predict(X.iloc[val_idx]),0,20)
        y_test += np.clip(clf.predict(X_test),0,20) / folds.n_splits
        score += rmse(np.clip(clf.predict(X.iloc[val_idx]),0,20), y[val_idx])
        print('Fold: {} score: {}'.format(i,rmse(np.clip(clf.predict(X.iloc[val_idx]),0,20), y[val_idx])))
    print('Avg 100* RMSE', score / folds.n_splits) 
        
    return y_oof, y_test 

In [52]:
y_oof, y_test_lgb = k_folds_lgb(train, y, test, k= 25,n_est=220,num_leaves = 48, lr = 0.205)

/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:741: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  .format(key))
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Fold: 0 score: 100.57272805783872
Fold: 1 score: 97.73064661837468
Fold: 2 score: 98.40178930877774
Fold: 3 score: 96.91782399070912
Fold: 4 score: 97.64109311773407
Fold: 5 score: 96.7767093182681
Fold: 6 score: 98.91219366858535
Fold: 7 score: 98.95567139678192
Fold: 8 score: 99.09917338860843
Fold: 9 score: 98.26680785309976
Fold: 10 score: 97.00461290252504
Fold: 11 score: 97.78534934874105
Fold: 12 score: 98.74408790764177
Fold: 13 score: 98.30291787769964
Fold: 14 score: 99.04026993588573
Fold: 15 score: 98.2130522169704
Fold: 16 score: 99.04379379896424
Fold: 17 score: 97.22407955134865
Fold: 18 score: 97.90462978719894
Fold: 19 score: 98.7598709695285
Fold: 20 score: 97.53394247378183
Fold: 21 score: 96.91715292642465
Fold: 22 score: 98.73922878267582
Fold: 23 score: 97.90726009528672
Fold: 24 score: 95.95823284851917
Avg 100* RMSE 98.09412472567881


In [43]:
#lowest #Avg RMSE 97.85571717415489
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

In [53]:
submit = pd.DataFrame({'reservation_id':test_id, 'amount_spent_per_room_night_scaled':y_test_lgb}, columns=['reservation_id', 'amount_spent_per_room_night_scaled'])
#submit.amount_spent_per_room_night_scaled = submit.amount_spent_per_room_night_scaled.astype('int')
#submit['num_orders'][submit['amount_spent_per_room_night_scaled'] <0] = 0
submit.to_csv('submit_klgb.csv', index=False)
create_download_link('submit_klgb.csv')

In [55]:
submit.head()

,reservation_id,amount_spent_per_room_night_scaled
341424,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d...,8.222305
341425,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb...,7.400039
341426,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e...,7.577269
341427,09593c907ae262e46f655b4db9e14f54a19eadcfdd2679...,7.913736
341428,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1...,7.743199


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold, KFold
def k_folds_rf(X, y, X_test, k,n_est,num_leaves,max_feat):
    folds = KFold( n_splits = k, shuffle=True, random_state=2)
    y_test = 0
    y_oof = np.zeros((X.shape[0]))
    score = 0
    for i, (train_idx, val_idx) in  enumerate(folds.split(X, y)):
        clf =  RandomForestRegressor(random_state =1, n_estimators = n_est, n_jobs = -1,min_samples_leaf=num_leaves,max_features = max_feat)
        clf.fit(X.iloc[train_idx], y[train_idx])
        y_oof[val_idx] = np.clip(clf.predict(X.iloc[val_idx]),0,20)
        y_test += np.clip(clf.predict(X_test),0,20) / folds.n_splits
        score += rmse(np.clip(clf.predict(X.iloc[val_idx]),0,20), y[val_idx])
        print('Fold: {} score: {}'.format(i,rmse(np.clip(clf.predict(X.iloc[val_idx]),0,20), y[val_idx])))
    print('Avg 100* RMSE', score / folds.n_splits) 
        
    return y_oof, y_test 

In [ ]:
y_oof, y_test_rf = k_folds_rf(train, y, test, k= 10,n_est=50,num_leaves = 20, max_feat ='sqrt')